In [1]:
%%capture
!pip install networkx

In [2]:
import networkx as nx

from enum import Enum
from itertools import chain, combinations
from typing import Set

# An Implementation of the Reachable Algorithm

By Alessio Zanga and Fabio Stella

## Implementation

Define an enumerator from UP/DOWN constants:

In [3]:
class Direction(Enum):
    DOWN = 0
    UP = 1

Implement the $Reachable$ algorithm:

In [4]:
def reachable(G: nx.DiGraph, X: str, Z: Set[str]) -> Set[str]:
    # Phase I: Insert all ancestors of Z into A.

    # Ancestral set of Z.
    A = set()
    # To-be-visited nodes set.
    L = Z.copy()
    # While there are still nodes to be visited.
    while L:
        # Select a node to be visited.
        Y = L.pop()
        # If it is not in the ancestral set.
        if Y not in A:
            # Add its parents to the nodes to be visited.
            L |= set(G.predecessors(Y))
            # Add it to ancestral set.
            A |= { Y }
    
    # Phase II: Traverse active trails starting from X.

    # Reachable nodes set.
    R = set()
    # Already-visited nodes set.
    V = set()
    # To-be-visited nodes set.
    L = { (X, Direction.UP) }
    # While there are still nodes to be visited.
    while L:
        # Select a node to be visited.
        (Y, d) = L.pop()
        # Check if already visited.
        if (Y, d) not in V:
            # If not in the conditioning set.
            if Y not in Z:
                # Then, it is reachable.
                R |= { Y }
            # Set (Y, d) as visisted.
            V |= { (Y, d) }
            # Trail up.
            if d == Direction.UP and Y not in Z:
                # For each parent of Y, visit from bottom.
                L |= { (W, Direction.UP) for W in G.predecessors(Y)}
                # For each child of Y, visit from top.
                L |= { (W, Direction.DOWN) for W in G.successors(Y) }
            # Trail down.
            elif d == Direction.DOWN:
                if Y not in Z:
                    # For each child of Y, visit from top.
                    L |= { (W, Direction.DOWN) for W in G.successors(Y) }
                if Y in A:
                    # For each parent of Y, visit from bottom.
                    L |= { (W, Direction.UP) for W in G.predecessors(Y)}
    
    # Return the reachable nodes.
    return R

Define a DAG $\mathcal{G}$:

In [5]:
G = nx.DiGraph([("A", "C"), ("B", "C"), ("C", "D"), ("A", "E"), ("B", "F")])

Define the powerset $2^{\mathbf{S}}$ of a set $\mathbf{S}$:

In [6]:
def powerset(S: Set[str]):
    return map(set, chain.from_iterable(
        combinations(S, k)
        for k in range(len(S) + 1)
    ))

list(powerset(["A", "B", "C"]))

[set(),
 {'A'},
 {'B'},
 {'C'},
 {'A', 'B'},
 {'A', 'C'},
 {'B', 'C'},
 {'A', 'B', 'C'}]

Test this $Reachable$ implementation for each possible combination of nodes given $\mathcal{G}$ using d-separation:

In [7]:
# Get the vertex set V.
V = set(G.nodes)
# For each X node in V.
for X in V:
    # For each node in 2^(V \ {X})
    for Z in powerset(V - { X }):
        # Get the reachable nodes from X given Z.
        R = reachable(G, X, Z)
        # For each node Y which is not X and not in Z.
        for Y in  V - { X } - Z:
            # Check if it is reachable and d-separated.
            is_reachable = (Y in R)
            is_d_separated = nx.is_d_separator(G, { X }, { Y }, Z)
            # If it is reachable then it is not d-separated, and vice-versa.
            assert is_reachable == (not is_d_separated)
            # Log the execution.
            print(f"Is '{Y}' reachable from '{X}' given '{Z}'? {is_reachable}")
            print(f"Is '{Y}' d-separated from '{X}' given '{Z}'? {is_d_separated}\n")

Is 'A' reachable from 'D' given 'set()'? True
Is 'A' d-separated from 'D' given 'set()'? False

Is 'E' reachable from 'D' given 'set()'? True
Is 'E' d-separated from 'D' given 'set()'? False

Is 'B' reachable from 'D' given 'set()'? True
Is 'B' d-separated from 'D' given 'set()'? False

Is 'F' reachable from 'D' given 'set()'? True
Is 'F' d-separated from 'D' given 'set()'? False

Is 'C' reachable from 'D' given 'set()'? True
Is 'C' d-separated from 'D' given 'set()'? False

Is 'F' reachable from 'D' given '{'A'}'? True
Is 'F' d-separated from 'D' given '{'A'}'? False

Is 'C' reachable from 'D' given '{'A'}'? True
Is 'C' d-separated from 'D' given '{'A'}'? False

Is 'E' reachable from 'D' given '{'A'}'? False
Is 'E' d-separated from 'D' given '{'A'}'? True

Is 'B' reachable from 'D' given '{'A'}'? True
Is 'B' d-separated from 'D' given '{'A'}'? False

Is 'F' reachable from 'D' given '{'E'}'? True
Is 'F' d-separated from 'D' given '{'E'}'? False

Is 'A' reachable from 'D' given '{'E'}'?